In [1]:
!nvidia-smi

Sat May 21 02:03:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 4.2 MB 7.4 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 6.6 MB 47.7 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 346 kB 8.1 MB/s 
     |████████████████████████████████| 1.1 MB 51.7 MB/s 
     |████████████████████████████████| 140 kB 60.1 MB/s 
     |████████████████████████████████| 212 kB 51.4 MB/s 
     |████████████████████████████████| 127 kB 41.0 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 47.7 MB/s 
     |████████████████████████████████| 271 kB 63.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
#transformers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

# Load Dataset


In [6]:
train_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/aug_clean_train.csv"
test_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_test.csv"

In [7]:
from datasets import Dataset, load_dataset

dataset = load_dataset('csv', data_files={'train': train_dataset_path,
                                          'test': test_dataset_path})

Using custom data configuration default-6d3dc670b2448c75


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6d3dc670b2448c75/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text_clean', 'Sentiment'],
        num_rows: 48483
    })
    test: Dataset({
        features: ['text_clean', 'Sentiment'],
        num_rows: 3787
    })
})

## Preprocess data

In [9]:
dataset = dataset.rename_column("text_clean", "text")
dataset = dataset.rename_column("Sentiment", "label")

In [10]:
def label_mapping(examples):
    label_to_idx = {'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                    'Positive': 2, 'Extremely Positive': 2}
    examples["label"] = label_to_idx[examples["label"]]
    return examples
    
dataset = dataset.map(label_mapping)

  0%|          | 0/48483 [00:00<?, ?ex/s]

  0%|          | 0/3787 [00:00<?, ?ex/s]

In [11]:
dataset['train'][0]

{'label': 1,
 'text': 'this says maybe a couple of days late reality is as thousands at our delivery point that arrive at least thirty minutes late... coronavirus restrictions hit you when people stop shopping online'}

In [12]:
def tokenizer_function(examples):
    result = tokenizer(examples["text"], max_length=128,
                       padding='max_length', truncation=True)
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets = dataset.map(tokenizer_function, batched=True, remove_columns=['text', 'label'])

  0%|          | 0/49 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 48483
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3787
    })
})

# Fine-tuning Roberta with Trainer API

In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [15]:
samples = [tokenized_datasets["train"][i] for i in range(2)]

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> <s>ont Gaul maybe a<mask> of days<mask> reality<mask> as thousands at our delivery point that arrive at least thirty minutes late... coronavirus<mask> hit you when people stop shopping online</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

'>>> <s><mask> supermarkets Cassandra upon people buying<mask>-essentials, reimbursement are they<mask><mask> Easter eggs at discount prices?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "/content/output",
    num_train_epochs=10,
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_strategy = "epoch",
    logging_steps=200
)

In [17]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator
)

In [18]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64


Perplexity: 36.66


In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 48483
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 7580


Epoch,Training Loss,Validation Loss
1,2.644800,2.243573
2,2.494500,2.205148
3,2.406800,2.105095
4,2.349300,2.146383
5,2.296300,2.093940
6,2.264000,2.014137
7,2.236100,2.039253
8,2.209400,2.039471
9,2.198300,2.052658
10,2.182200,2.047486


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64


Saving model checkpoint to /content/output/checkpoint-758
Configuration saved in /content/output/checkpoint-758/config.json
Model weights saved in /content/output/checkpoint-758/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/checkpoint-1516
Configuration saved in /content/output/checkpoint-1516/config.json
Model weights saved in /content/output/checkpoint-1516/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/checkpoint-2274
Configuration saved in /content/output/checkpoint-2274/config.json
Model weights saved in /content/output/checkpoint-2274/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/checkpoint-3032
Configuration saved in /content/output/checkpoint-3032/config.json
Model weights saved in /content/output/checkpoint-3032/pytorch_mode

TrainOutput(global_step=7580, training_loss=2.3378642099828393, metrics={'train_runtime': 7872.2536, 'train_samples_per_second': 61.587, 'train_steps_per_second': 0.963, 'total_flos': 3.190974864244992e+16, 'train_loss': 2.3378642099828393, 'epoch': 10.0})

In [20]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64


Perplexity: 7.66


In [21]:
model.save_pretrained('/content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/')

Configuration saved in /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model weights saved in /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/pytorch_model.bin
